In [1]:
import os


In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Chandu\\WorkSpace\\Learnings\\Repos\\End-to-End-TitanicSurvivalProject'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [10]:
from titanicSurvival.constants import *
from titanicSurvival.utils.common import  read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path= CONFIG_FILE_PATH,
                 params_file_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    def get_data_ingestion(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from titanicSurvival.logging import logger
from titanicSurvival.utils.common import get_size


In [13]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config =config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"File already exists with size: {get_size(self.config.local_data_file)}")

    
    def extract_zip_file(self):
        os.makedirs(self.config.unzip_dir,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file) as zf:
            zf.extractall(self.config.unzip_dir)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-29 17:01:56,810: INFO: common: yaml file: config\config.yaml loaded succesfully..]
[2025-03-29 17:01:56,814: INFO: common: yaml file: params.yaml loaded succesfully..]
[2025-03-29 17:01:56,817: INFO: common: Create Directory at :artifacts]
[2025-03-29 17:01:56,820: INFO: common: Create Directory at :artifacts/data_ingestion]
[2025-03-29 17:01:57,811: INFO: 3993488486: artifacts/data_ingestion/titanic.zip download! with following info: 
 Connection: close
Content-Length: 34877
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "26dabd81decc314fa4319d7243b78a1a271a50a2322be34002769e64e00f0a6a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F887:2BA4DF:3FAACD:9FB6C6:67E7D9D6
Accept-Ranges: bytes
Date: Sat, 29 Mar 2025 11:30:31 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100032-HYD
X-Cach